# PHISHES Digital Platform - Data Download Module

This notebook downloads and analyzes geospatial datasets from Azure Blob Storage for catchment-scale soil science modeling.

## Workflow Overview
1. **Step 1: Setup and Imports**: Import libraries and configure logging
2. **Step 2: Configure Paths**: Define catchment shapefile and project settings
3. **Step 3: Initialize**: Create project structure and setup downloader
4. **Step 4: Inspect Catchment**: Load and visualize study area boundary
5. **Step 5: Initialize Downloader**: Connect to Azure and configure the downloader
6. **Step 6: Browse Datasets**: List available categories and subcategories
7. **Step 7: Download Data**: Fetch climate, soil, or topography datasets clipped to catchment
8. **Step 8: Visualize**: Create spatial maps and temporal plots
9. **Step 9: Analyze**: Compute basin averages and time series statistics
10. **Step 10: Review Logs**: Inspect download history and metadata

## 📋 Step 1: Setup and Imports

**Imports all required libraries and configures logging.**

This cell:
- Imports core functionality (folder creation, downloader, data loading)
- Imports analysis functions (catchment processing, basin averaging)
- Imports visualization tools (spatial maps, time series plots)
- Configures logging to track download progress and errors
- Suppresses unnecessary warnings

✅ Run this cell first before proceeding.

In [ ]:
import json
import time
import warnings
from pathlib import Path

import matplotlib.pyplot as plt

from src.analysis import (
    compute_anomalies,
    compute_basin_average,
    compute_catchment_weights,
    load_catchment,
    plot_catchment,
    plot_spatial_map,
    plot_time_series,
)
from src.core import (
    PDPDataDownloader,
    build_dataset_path,
    cleanup_existing_dataset,
    create_pdp_folders,
    open_dataset_any,
)

warnings.filterwarnings("ignore")

## ⚙️ Step 2: Configuration

✅ **Edit the paths below to match your setup.**

This cell defines:
- Project name and base directory for downloaded data
- Catchment file path (Shapefile `.shp` or GeoJSON `.geojson`)
- Time range for temporal datasets (e.g., climate data)
- Output format preference (NetCDF, Zarr, or DFS2)
- Whether to mask data to the exact catchment shape (`True`) or keep the full bounding-box extent (`False`)

Editable parameters:
- `PROJECT_MAIN`: Short project identifier used in folder names
- `PROJECT_NAME`: Final project name (can be the same as `PROJECT_MAIN`)
- `PROJECT_BASE`: Base output directory for downloads. Default: sub-directory `data`
- `CATCHMENT_FILE`: Path to your catchment file (`.shp` or `.geojson`)
- `MASK_ON_CATCHMENT`: `True` to mask to catchment boundary, `False` to keep bounding box
- `CATCHMENT_EXTENT`: Optional manual extent `[minx, miny, maxx, maxy]`
- `CATCHMENT_CRS`: CRS for the manual extent (e.g., `EPSG:4326`)
- `TIME_RANGE`: Optional time range tuple `(start_date, end_date)`
- `OUTPUT_FORMAT`: Output format (`nc`, `zarr`, or `dfs2`)

In [ ]:
# Project Configuration
PROJECT_MAIN = ""  # str: short project identifier, used in folder names
PROJECT_NAME = f"{PROJECT_MAIN}"  # str: final project name (can be same as PROJECT_MAIN)
PROJECT_BASE = Path(f"../data/{PROJECT_NAME}")  # Path: base output directory for downloads

# Path to your catchment file (Shapefile .shp or GeoJSON .geojson)
CATCHMENT_FILE = Path(r"")
MASK_ON_CATCHMENT = True  # Whether to mask datasets to the catchment boundary (True/False)

# Optional manual extent (use instead of CATCHMENT_FILE)
CATCHMENT_EXTENT = None  # [minx, miny, maxx, maxy]
CATCHMENT_CRS = "EPSG:4326"  # CRS for the extent

# Time range for temporal datasets (optional)
TIME_RANGE = ("2015-01-01", "2020-12-31")  # Adjust as needed

# Output format ("nc", "zarr", or "dfs2")
OUTPUT_FORMAT = "dfs2"

print(f"Project base:   {PROJECT_BASE}")
print(f"Catchment:      {CATCHMENT_FILE}")
print(f"Mask dataset:   {MASK_ON_CATCHMENT}")
print(f"Time range:     {TIME_RANGE}")
print(f"Output format:  {OUTPUT_FORMAT}")

## 🔧 Step 3: Initialize Project Structure

**Creates the project folder hierarchy for organized data management.**

This cell:
- Creates base project directory if it doesn't exist
- Creates `logs/` subdirectory for download tracking
- Displays confirmation of project initialization
- Data folders (climate/, soil/, etc.) are created automatically during downloads

### Initial structure

```
<your_project>/
└── logs/
    └── download_log.json
```

### After downloading datasets

```
<your_project>/
├── data/
│   └── <category>/
│       └── <subcategory>/
│           └── <subcategory>.<format>
└── logs/
    └── download_log.json
```

In [ ]:
# Initialize project (creates base folder and logs directory)
print("Initializing PDP project...")
base_path = create_pdp_folders(base_path=PROJECT_BASE)

print(f"Project initialized at: {base_path}")
print("Data folders will be created automatically when downloading datasets.")

## 📍 Step 4: Load and Visualize Catchment

**Read catchment shapefile and display study area boundary.**

This cell:
- Loads catchment polygon from shapefile (loaded once, reused throughout)
- Supports **polygons**, **lines**, and **points** (lines/points are buffered to polygons)
- Automatically reprojects to EPSG:4326 (lat/lon) if needed
- Merges multiple features into single polygon if present
- **Validates European AOI**: Checks catchment overlaps with Europe
- **Validates size**: Ensures area is within 0.01 - 500,000 km²
- Displays catchment bounds (min/max lat/lon)
- Creates visualization map of catchment boundary

**Alternative:** You can use a manual extent and projection (bbox + CRS) instead of a polygon file.

In [ ]:
# Load and visualize catchment (loaded once here, reused throughout notebook)
# Supports polygons, lines, and points (lines/points are buffered to polygons)
# Validates: European AOI overlap, catchment size limits

if CATCHMENT_EXTENT is not None:
    catchment, catchment_geom = load_catchment(
        extent=CATCHMENT_EXTENT,
        extent_crs=CATCHMENT_CRS,
        target_crs="EPSG:4326",
    )
else:
    catchment, catchment_geom = load_catchment(CATCHMENT_FILE, target_crs="EPSG:4326")

print(f"Catchment bounds: {catchment.total_bounds}")

fig, ax = plot_catchment(catchment, title="Catchment Boundary")
plt.show()

## 🌐 Step 5: Initialize Data Downloader

**Setup Azure connection and configure downloader with catchment.**

This cell:
- Uses the catchment geometry loaded in Step 4 (no redundant file read)
- Initializes connection to Azure Blob Storage using credentials
- Sets buffer distance (in grid cells) around catchment (default: 1 cell)
- Specifies output format (NetCDF or Zarr)
- Tests connection and displays any errors with troubleshooting tips

Editable parameters:
- `catchment`: GeoDataFrame loaded in Step 4
- `output_base`: Base output directory (uses `PROJECT_BASE`)
- `buffer_cells`: Number of grid cells to buffer around the catchment
- `output_format`: Output format (uses `OUTPUT_FORMAT`)
- `mask_on_catchment`: `True` to mask to catchment boundary, `False` to keep bounding box

In [ ]:
# Initialize downloader (uses catchment loaded in Step 3)
print("Initializing data downloader...")

try:
    downloader = PDPDataDownloader(
        catchment=catchment,  # Use pre-loaded GeoDataFrame from Step 3
        output_base=PROJECT_BASE,
        buffer_cells=1,  # Buffer in grid cells
        output_format=OUTPUT_FORMAT,
        mask_on_catchment=MASK_ON_CATCHMENT,
    )
    print("Downloader initialized successfully")

except Exception as e:
    print(f"ERROR: Failed to initialize downloader: {e}")
    print("Please check:")
    print("1. Catchment was loaded in Step 3")
    print("2. Azure credentials are configured")
    print("3. You have internet connection")

## 📚 Step 6: Browse Available Datasets

**Query Azure storage to see what datasets are available for download.**

This cell:
- Queries the dataset catalog from Azure storage
- Lists available categories (climate, soil, topography, landuse, hydrology)
- Displays subcategories and their descriptions
- Shows available variables for each dataset
- Helps you decide what to download

Editable parameters:
- `catalog`: Dataset catalog returned by `downloader.list_available_datasets()`
- `Category/Subcategory`: Choose values to use in Step 7.A

In [ ]:
# List available datasets
catalog = downloader.list_available_datasets()

print("AVAILABLE DATASETS")
print("=" * 70)
print(f"{'Category':<15} {'Subcategory':<25} {'Description':<30}")
print("-" * 70)

for category, subcategories in catalog.items():
    for subcategory, info in subcategories.items():
        print(f"{category:<15} {subcategory:<25} {info['description']:<30}")

print("=" * 70)
print("\n→ Copy the category and subcategory values to Step 6.A below.")

## ⬇️ Step 7: Download Dataset

**Download a specific dataset clipped to your catchment extent.**

### Step 7.A: Configure Download

Specify which dataset to download and clean up any existing data.

In [ ]:
# Configure which dataset to download/clean up
DATASET_CATEGORY = "climate"  # e.g., "climate"
DATASET_SUBCATEGORY = "era5_precipitation"  # e.g., "era5_precipitation", "era5_temperature"


# Build path automatically based on selection
dataset_path = build_dataset_path(
    PROJECT_BASE, DATASET_CATEGORY, DATASET_SUBCATEGORY, OUTPUT_FORMAT
)

print(f"Target dataset: {DATASET_CATEGORY}/{DATASET_SUBCATEGORY}")
print(f"Path: {dataset_path}")

# Close open datasets, force GC, and remove existing data if present
cleanup_existing_dataset(dataset_path, namespace=locals())

### Step 7.B: Execute Download

**Main download operation with automatic retries and verification.**

This cell:
- Streams data from Azure Blob Storage
- Clips spatially to catchment bounds (with buffer)
- Applies temporal filtering based on TIME_RANGE
- Saves locally in specified format (NetCDF or Zarr)
- Verifies downloaded data dimensions and variables
- Displays data summary (time range, spatial extent, dimensions)

Editable parameters (edit these in Step 7.A or Step 2):
- `DATASET_CATEGORY`: Dataset category to download (e.g., `climate`)
- `DATASET_SUBCATEGORY`: Dataset subcategory (e.g., `era5_precipitation`)
- `TIME_RANGE`: Optional time range tuple `(start_date, end_date)`
- `variables`: List of variable names to download (set in code as `None` for all)
- `OUTPUT_FORMAT`: Output format set in Step 2 (`nc`, `zarr`, or `dfs2`)

**Note:** On Windows, if you get "file is being used" errors, run Step 7.A above first to close open datasets.

In [ ]:
# Download dataset
try:
    start_time = time.time()

    output_path = downloader.download_dataset(
        category=DATASET_CATEGORY,
        subcategory=DATASET_SUBCATEGORY,
        time_range=TIME_RANGE,
        variables=None,  # Download all variables
    )

    download_time = time.time() - start_time

    # Verify the download using the same path builder
    ds_downloaded = open_dataset_any(output_path)
    print("Dataset verification:")
    print(f"  Dimensions: {dict(ds_downloaded.dims)}")
    print(f"  Variables: {list(ds_downloaded.data_vars)}")

    if "time" in ds_downloaded.dims:
        print(
            f"  Time range: {str(ds_downloaded.time.min().values)[:10]} to {str(ds_downloaded.time.max().values)[:10]}"
        )

    if "lat" in ds_downloaded.dims and "lon" in ds_downloaded.dims:
        if ds_downloaded.dims["lat"] > 0 and ds_downloaded.dims["lon"] > 0:
            print(
                f"  Lat range: {float(ds_downloaded.lat.min()):.2f}° to {float(ds_downloaded.lat.max()):.2f}°"
            )
            print(
                f"  Lon range: {float(ds_downloaded.lon.min()):.2f}° to {float(ds_downloaded.lon.max()):.2f}°"
            )

    print(f"\nDownload completed in {download_time:.1f} seconds")

except Exception as e:
    print(f"ERROR: Download failed: {e}")
    print("Note: Ensure Azure credentials are properly configured")
    print("On Windows, if file is locked: Run the cell above first, then retry")

### Step 7.C: Batch Download (Optional)

**Download all available datasets at once.**

Use `download_all()` instead of downloading datasets one at a time. This will iterate through the full catalog and download each dataset clipped to your catchment.

In [ ]:
# Batch download all available datasets (optional)
# downloader.download_all(time_range=TIME_RANGE)

## 🗺️ Step 8: Visualize Spatial Data

**Create spatial map of downloaded dataset with catchment overlay.**

This cell:
- Loads the downloaded dataset from local storage
- Extracts first variable and first timestep (if temporal)
- Loads catchment boundary for overlay
- Creates spatial map with catchment outline
- Displays variable name, dataset category, and date (if applicable)

Editable parameters:
- `DATASET_CATEGORY`: Dataset category used to build the path
- `DATASET_SUBCATEGORY`: Dataset subcategory used to build the path
- `OUTPUT_FORMAT`: Output format used to build the path
- `TIMESTEP`: `None`, integer index, or date string for time selection

In [ ]:
# Visualize spatial data with catchment overlay
dataset_path = build_dataset_path(
    PROJECT_BASE, DATASET_CATEGORY, DATASET_SUBCATEGORY, OUTPUT_FORMAT
)

# Configure timestep selection (set to None for first timestep, or specify index/date)
TIMESTEP = (
    None  # Options: None (first), integer index (e.g., 0, 10, -1), or date string ("2018-06-15")
)

if dataset_path.exists():
    ds = open_dataset_any(dataset_path)
    print(f"Loaded: {dict(ds.dims)}, Variables: {list(ds.data_vars)}")

    # Get first variable
    var_name = list(ds.data_vars)[0]
    data = ds[var_name]

    # Select timestep
    if "time" in data.dims:
        if TIMESTEP is None:
            data_plot = data.isel(time=0)
            time_label = f" ({str(data.time[0].values)[:10]})"
        elif isinstance(TIMESTEP, int):
            data_plot = data.isel(time=TIMESTEP)
            time_label = f" ({str(data.time[TIMESTEP].values)[:10]})"
        elif isinstance(TIMESTEP, str):
            data_plot = data.sel(time=TIMESTEP, method="nearest")
            time_label = f" ({str(data_plot.time.values)[:10]})"
        print(
            f"Available time range: {str(data.time.min().values)[:10]} to {str(data.time.max().values)[:10]}"
        )
        print(f"Selected timestep: {time_label.strip(' ()')}")
    else:
        data_plot = data
        time_label = ""

    # Use catchment loaded in Step 3
    fig, ax = plot_spatial_map(
        data_plot,
        catchment=catchment,
        title=f"{DATASET_CATEGORY}/{DATASET_SUBCATEGORY}: {var_name}{time_label}",
    )
    plt.show()
else:
    print(f"ERROR: Dataset not found at: {dataset_path}")

## 📊 Step 9: Time Series Analysis

**Compute area-weighted basin average and extract time series.**

This cell:
- Loads catchment polygon and downloaded dataset
- Computes area-weighted intersection fractions for each grid cell
- Calculates basin-averaged time series (properly weighted)
- Plots time series showing temporal evolution
- Computes anomaly statistics (mean, std, percentiles)

**Operations:**
- Basin averaging accounts for partial grid cell overlap
- Time series represents catchment-scale aggregate values
- Anomalies show deviations from long-term mean

Editable parameters:
- `DATASET_CATEGORY`: Dataset category used to build the path
- `DATASET_SUBCATEGORY`: Dataset subcategory used to build the path
- `OUTPUT_FORMAT`: Output format used to build the path

In [ ]:
# Compute area-weighted basin time series
dataset_path = build_dataset_path(
    PROJECT_BASE, DATASET_CATEGORY, DATASET_SUBCATEGORY, OUTPUT_FORMAT
)

# Use catchment loaded in Step 3
ds = open_dataset_any(dataset_path)

var_name = list(ds.data_vars)[0]
data = ds[var_name]

print(f"Variable: {var_name}, Shape: {dict(ds.dims)}")

# Compute weights and basin average (using catchment_geom from Step 3)
weights = compute_catchment_weights(ds, catchment_geom) # type: ignore
basin_avg = compute_basin_average(data, weights)

# Plot time series
if "time" in data.dims:
    # Compute anomalies first and print stats above the plot
    anomaly_stats = compute_anomalies(basin_avg)

    fig, ax = plot_time_series(
        basin_avg,
        label="Basin Average (Area-Weighted)",
        title=f"Area-Weighted Basin {var_name} Time Series",
        ylabel=var_name,
    )
    plt.show()

## 📝 Step 10: Review Download History

**Check download log to see what has been downloaded and when.**

This cell:
- Reads the download log JSON file
- Displays download history as formatted table
- Shows timestamp, dataset, status (success/error), and file size
- Summarizes total downloads and success rate
- Useful for tracking data provenance and debugging

**Note:** Log file is created after first successful download.

In [ ]:
# Check download history
log_file = PROJECT_BASE.joinpath("logs", "download_log.json")

if log_file.exists():
    try:
        with open(log_file, "r") as f:
            log_data = json.load(f)

        # Handle both dict and list formats
        downloads = log_data.get("downloads", []) if isinstance(log_data, dict) else log_data

        print("Download History:")
        print("=" * 80)
        for i, entry in enumerate(downloads, 1):
            print(f"\n{i}. {entry.get('category', 'N/A')}/{entry.get('subcategory', 'N/A')}")
            print(f"   Timestamp: {entry.get('timestamp', 'N/A')}")
            print(f"   Output: {entry.get('output_path', 'N/A')}")
            if entry.get("time_range"):
                print(f"   Time range: {entry['time_range']}")

        print(f"\nTotal downloads logged: {len(downloads)}")

    except Exception as e:
        print(f"WARNING: Could not parse log file: {e}")
else:
    print(f"No download log found at: {log_file}")
    print("The log file will be created after the first download.")